In [21]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
images_path=r'images/'
import os
import time
import pickle
import pandas as pd

In [22]:
def write_ply(fn, verts, colors):
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    out_colors = colors.copy()
    verts = verts.reshape(-1, 3)
    verts = np.hstack([verts, out_colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [23]:
pos_df=pd.read_csv('Data/imu_pos_output.csv')

In [24]:
img_paths_list=[str(images_path+"image"+str(i)+" .jpg") for i in range (len(os.listdir(images_path))-1) ]
print(len(img_paths_list))
file = open('VisualOdometry/cameraMatrix.pkl', 'rb')
data = pickle.load(file)
file.close()

764


In [32]:
for i in range(1,len(img_paths_list)+1):
  
  imgL=cv2.imread(img_paths_list[i-1],cv2.IMREAD_GRAYSCALE)
  img_left_color=cv2.imread(img_paths_list[i-1])
  imgR=cv2.imread(img_paths_list[i],cv2.IMREAD_GRAYSCALE)
  
  stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
  disparity = stereo.compute(imgL,imgR)
  norm_image = cv2.normalize(disparity, None, alpha = 0, beta = 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  norm_image=cv2.resize(norm_image,(1280,720))
  cam1 = data[0]
  cam2 = data[0]
  rev_proj_matrix = np.zeros((4,4)) # to store the output

  t_matrix = np.array(
    [
      pos_df['x'][i]-pos_df['x'][i-1],
      pos_df['y'][i]-pos_df['y'][i-1],
      pos_df['z'][i]-pos_df['z'][i-1]
    ])
  
  print(cam2)
  
  cv2.stereoRectify(cameraMatrix1 = cam1 ,cameraMatrix2 = cam2,
                  distCoeffs1 = 0, distCoeffs2 = 0,
                  imageSize = imgL.shape,
                  R = np.identity(3), T = t_matrix,
                  R1 = None, R2 = None,
                  P1 =  None, P2 =  None, 
                  Q = rev_proj_matrix)
  
  points = cv2.reprojectImageTo3D(disparity, rev_proj_matrix)
  colors = cv2.cvtColor(img_left_color, cv2.COLOR_BGR2RGB)
  colors = colors.resize(-1, 3)
  write_ply('Data/out.ply',points, colors)
  cv2.imshow("disp_map",norm_image)
  cv2.waitKey(2)

cv2.destroyAllWindows()


[5630.84630351    0.         3559.54529106]


error: OpenCV(4.7.0) /io/opencv/modules/calib3d/src/undistort.dispatch.cpp:403: error: (-215:Assertion failed) CV_IS_MAT(_cameraMatrix) && _cameraMatrix->rows == 3 && _cameraMatrix->cols == 3 in function 'cvUndistortPointsInternal'
